In [ ]:
import numpy as np
import os.path as osp
import cv2
import time
import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras.backend as K
%matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload 2

from modules.images_viewer import ImagesViewer
from modules.dataset import Dataset
from modules.detector import FCNDetector
from modules.images_viewer import ImagesViewer
from modules.quality import compute_quality, compute_average_precision, find_optimal_threshold
import modules.models.loader as loader
import modules.models.simple_model
import config

In [ ]:
def show(image, size_w=8, size_h=8):
    image = image[:, :, ::-1]
    plt.figure(figsize=(size_w, size_h))
    plt.imshow(image)

# Input

In [ ]:
detector = FCNDetector()

In [ ]:
dataset = Dataset()

# Show image with rects

In [ ]:
image_data = dataset.images_data[0]
if not config.load_all_images_to_ram:
    image_data.load()
image = image_data.image.copy()
for true_rect in image_data.rects:
    true_rect.draw(image, (0, 255, 0), 5)

mask = cv2.resize(image_data.mask, (0, 0), fx=1.0/config.mask_downsample_rate, fy=1.0/config.mask_downsample_rate)
nms_heat_map = detector.heat_map_nms(mask)
rects = detector.obtain_rects(nms_heat_map, mask)
reduced_rects = FCNDetector.rects_nms(rects)
for rect in reduced_rects:
    rect.draw(image, (255, 0, 0), 3)
print(len(reduced_rects))
show(image, 22, 22)
if not config.load_all_images_to_ram:
    image_data.release()

# Check NMS

In [ ]:
for image_data in tqdm.tqdm_notebook(dataset.images_data):
    if not config.load_all_images_to_ram:
        image_data.load()
    mask = cv2.resize(image_data.mask, (0, 0), fx=1.0/config.mask_downsample_rate, fy=1.0/config.mask_downsample_rate)
    nms_heat_map = detector.heat_map_nms(mask)
    rects = detector.obtain_rects(nms_heat_map, mask)
    reduced_rects = FCNDetector.rects_nms(rects)
    if len(image_data.rects) == len(reduced_rects):
        print("{}:\t ok".format(image_data.image_name))
    else:
        print("{}:\t missmatch {} != {}".format(image_data.image_name, len(image_data.rects), len(reduced_rects)))
        
    if not config.load_all_images_to_ram:
        image_data.release()

# Check Quality

In [ ]:
quality_objects_union = []
for image_data in tqdm.tqdm_notebook(dataset.images_data):
    if not config.load_all_images_to_ram:
        image_data.load()
    mask = cv2.resize(image_data.mask, (0, 0), fx=1.0/config.mask_downsample_rate, fy=1.0/config.mask_downsample_rate)
    nms_heat_map = detector.heat_map_nms(mask)
    rects = detector.obtain_rects(nms_heat_map, mask)
    reduced_rects = FCNDetector.rects_nms(rects)
    quality_objects = compute_quality(image_data.rects, reduced_rects)
    quality_objects_union.extend(quality_objects)
    ap_rate = compute_average_precision(quality_objects)
    th, best_precision, best_recall, best_f1 = find_optimal_threshold(quality_objects)
    
    if ap_rate > 0.9999 and best_precision > 0.9999 and best_recall > 0.9999:
        print("{}:\t ok".format(image_data.image_name))
    else:
        print("{}:\t missmatch {}".format(image_data.image_name, best_recall))
        
    if not config.load_all_images_to_ram:
        image_data.release()

In [ ]:
true_labels = []
predictions = []
for q in quality_objects:
    true_labels.append(q.label)
    predictions.append(q.prediction)
precision, recall, thresholds = precision_recall_curve(true_labels, predictions)

In [ ]:
recall

In [ ]:
find_optimal_threshold(quality_objects_union)

# Estimate Quality

In [ ]:
K.clear_session()
fcn_model_module = loader.get_fcn_model_module()
fcn_model = fcn_model_module.FCNModel()
detector = FCNDetector(fcn_model.model, osp.join(fcn_model.weights_dir, 'best_weights.hdf5'))

### One Image Processing

In [ ]:
image = dataset.images_data[0].image
mask = detector.predict_heat_maps_batch(np.asarray([image]))[0]

In [ ]:
mask[:, :, 0].max()

In [ ]:
show(mask, 26, 26)

In [ ]:
nms_heat_map = detector.heat_map_nms(mask)

In [ ]:
rects = detector.obtain_rects(nms_heat_map, mask)
reduced_rects = FCNDetector.rects_nms(rects)

In [ ]:
quality_objects = compute_quality(dataset.images_data[0].rects, reduced_rects)

In [ ]:
ap_rate = compute_average_precision(quality_objects)
ap_rate

In [ ]:
threshold, f1_score = find_optimal_threshold(quality_objects)

In [ ]:
threshold, f1_score

In [ ]:
filtered_rects = [rect for rect in reduced_rects if rect.score > threshold]
len(filtered_rects)

In [ ]:
quality_objects1 = compute_quality(dataset.images_data[0].rects, filtered_rects)

In [ ]:
len(dataset.images_data[0].rects)

In [ ]:
len(reduced_rects)

In [ ]:
images_viewer = ImagesViewer()
images_viewer.set_images([image, mask*255, nms_heat_map*255])
images_viewer.run()

In [ ]:
cv2.destroyAllWindows()

### Quality over the all images

In [ ]:
def get_quality_objects(image_data):
    if not config.load_all_images_to_ram:
        image_data.load()
    mask = detector.predict_heat_maps_batch(np.asarray([image_data.image]))[0]
    nms_heat_map = detector.heat_map_nms(mask)
    rects = detector.obtain_rects(nms_heat_map, mask)
    reduced_rects = FCNDetector.rects_nms(rects)
    quality_objects = compute_quality(image_data.rects, reduced_rects)
    ap_rate = compute_average_precision(quality_objects)
    
    print("{}:\t {}".format(image_data.image_name, ap_rate))

    if not config.load_all_images_to_ram:
        image_data.release()
        
    return quality_objects

In [ ]:
train_quality_objects = []
for i in tqdm.tqdm_notebook(dataset.train_indices):
    image_data = dataset.images_data[i]
    train_quality_objects.extend(get_quality_objects(image_data))

test_quality_objects = []
for i in tqdm.tqdm_notebook(dataset.test_indices):
    image_data = dataset.images_data[i]
    test_quality_objects.extend(get_quality_objects(image_data))    

train_ap_rate = compute_average_precision(train_quality_objects)
print("Train AP: {}".format(train_ap_rate))
test_ap_rate = compute_average_precision(test_quality_objects)
print("Test AP: {}".format(test_ap_rate))

threshold, best_precision, best_recall, best_f1 = find_optimal_threshold(train_quality_objects)
print("Threshold {}, F1 score {}".format(threshold, f1_score))

In [ ]:
threshold, best_precision, best_recall, best_f1 = find_optimal_threshold(train_quality_objects)
print("Threshold {}, F1 score {}, precision {}, recall {}".format(threshold, best_f1, best_precision, best_recall))

In [ ]:
threshold, best_precision, best_recall, best_f1 = find_optimal_threshold(test_quality_objects)
print("Threshold {}, F1 score {}, precision {}, recall {}".format(threshold, best_f1, best_precision, best_recall))

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

def f1(precision, recall):
    return 2*(precision*recall)/(precision + recall)


true_labels = []
predictions = []
for q in train_quality_objects:
    true_labels.append(q.label)
    predictions.append(q.prediction)
precision, recall, thresholds = precision_recall_curve(true_labels, predictions)
f1_scores = [(th, f1(precision[i], recall[i])) for i, th in enumerate(thresholds)]
f1_scores = sorted(f1_scores, key=lambda x: -x[1])

In [ ]:
for i in range(len(thresholds)):
    print(thresholds[i], precision[i+1], recall[i+1])

In [ ]:
1

In [ ]:
len(precision)

In [ ]:
precision, recall, thresholds = precision_recall_curve(true_labels, predictions)
f1_scores = [(th, f1(precision[i+1], recall[i+1])) for i, th in enumerate(thresholds)]
f1_scores = sorted(f1_scores, key=lambda x: -x[1])

In [ ]:
1